In [ ]:
## Load in the data
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
import tensorflow
import keras
%matplotlib inline

In [ ]:
columns=['seconds','mic (trachea)','abdominal movement (stretch sensor)','thorax movement (stretch sensor)']
data_test=['seconds','thorax movement (stretch sensor)','abdominal movement (stretch sensor)','mic (right of trachea)','mic 2 (left of trachea)','mic (trachea)',]

In [ ]:
data = np.load('overnight_all_compressed.npz')


In [ ]:
final_array=np.array([])

a = data['arr_0']
b = data['arr_1']
c = data['arr_2']
d= c[:,(0,5,2,1)]
print(a.shape)
print(b.shape)


final_array=np.vstack([b,d])

print(final_array.shape)


In [ ]:
df = pd.DataFrame(final_array,columns=columns)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
X_train = final_array[:,1]
Y_train = final_array[:,2]
l= len(X_train)
Xtrain = X_train[:round(l*0.7)]
Xtest = X_train[round(l*0.7)+1:]
ytrain = Y_train[:round(l*0.7)]
ytest = Y_train[round(l*0.7)+1:]

http://barnesanalytics.com/analyzing-multivariate-time-series-using-arimax-in-python-with-statsmodels 


http://barnesanalytics.com/basics-of-arima-models-with-statsmodels-in-python

In [ ]:
import statsmodels.api as sm
print(sm.tsa.stattools.adfuller(df['mic (trachea)'],autolag=None,maxlag=0))
print(sm.tsa.stattools.adfuller(df['abdominal movement (stretch sensor)'],autolag=None,maxlag=0))


In [ ]:
df['const']=1 
model1=sm.OLS(endog=df['mic (trachea)'],exog=df[['abdominal movement (stretch sensor)','const']])
results1=model1.fit()
print(results1.summary())


In [ ]:
df['diffS']=df['mic (trachea)'].diff() 
df['diffM']=df['abdominal movement (stretch sensor)'].diff() 
model2=sm.OLS(endog=df['diffS'].dropna(),exog=df[['diffM','const']].dropna()) 
results2=model1.fit() 
print(results2.summary())


In [ ]:
print(sm.tsa.stattools.grangercausalitytests(df[['abdominal movement (stretch sensor)','mic (trachea)']].dropna(),1))

In [ ]:
df['lag']=df['diffM'].shift()
df.dropna(inplace=True)
model3=sm.tsa.ARIMA(endog=df['Sales'],exog=df[['lag']],order=[1,1,0])
results3=model3.fit()
print(results3.summary())1

In [ ]:
print(Xtrain.shape)
print(ytrain.shape)

#Xtrain=Xtrain.reshape(-1,1)
#ytrain=ytrain.reshape(-1,1)
Xtest=Xtest.reshape(-1,1)

In [ ]:
window_size = 1000
leftover = len(Xtrain)%window_size

X_reshaped_train=(Xtrain[:-leftover]).reshape(-1,window_size,1)
print(X_reshaped_train.shape)

y_reshaped_train=(ytrain[:-leftover]).reshape(-1,window_size,1)

leftover_test = len(Xtest)%window_size

X_reshaped_test=(Xtest[:-leftover_test]).reshape(-1,window_size,1)
y_reshaped_test=(ytest[:-leftover_test]).reshape(-1,window_size,1)


In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(Xtrain,ytrain)
y_pred = regr.predict(Xtest)

In [ ]:
plt.plot(ytest,  color='black')
plt.plot(y_pred, color='blue')


In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4,  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)


# Xtrain=(Xtrain).reshape(len(Xtrain),1,1)

# ytrain=ytrain.reshape(len(ytrain),1,1)

# Xtest=Xtest.reshape(len(Xtest),1,1)

# ytest=ytest.reshape(len(ytest),1,1)

validationData=(Xtest,ytest)
'''
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)'''
print(Xtrain.shape)

model=baseline_model()
history = model.fit(X_reshaped_train,y_reshaped_train,
                    validation_data=(X_reshaped_test, y_reshaped_test),
                    epochs=3,batch_size=50,
                    callbacks=[early_stopping])

In [ ]:
y_pred=model.predict(X_reshaped_train)
y_pred_unravel=y_pred.reshape(-1,1)
print(y_pred_unravel.shape)

In [ ]:
y_pred=model.predict(X_reshaped_train)
y_pred.reshape(-1,1)
print(y_pred.shape)
y_pred= y_pred.reshape(-1,1)
print(y_pred)
fig = plt.figure(figsize=(20,10))
#for ix,items in zip([1,2],x):
ax = fig.add_subplot(2,1,1)
ax.set_title('Time vs Patient Data')
ax.set_xlabel('Time(in sec)')
ax.set_ylabel('Patient Data')
ax.plot(y_pred,label='abdominal movement (stretch sensor)_pred')
ax.plot(y_reshaped_train.reshape(-1,1),label='abdominal movement (stretch sensor)_true')
#ax.plot(final_array[:,2],label='abdominal movement (stretch sensor)_original')




ax.legend()


In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=2, random_state=seed)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
#http://barnesanalytics.com/analyzing-multivariate-time-series-using-arimax-in-python-with-statsmodels import statsmodels.api as sm print(sm.tsa.stattools.adfuller(df['Marketing'])) print(sm.tsa.stattools.adfuller(df['Sales']))
df['const']=1 model1=sm.OLS(endog=df['Sales'],exog=df['Marketing','const']) results1=model1.fit() print(results1.summary())
df['diffS']=df['Sales'].diff() df['diffM']=df['Marketing'].diff() model2=sm.OLS(endog=df['diffS'].dropna(),exog=df'diffM','const'.dropna()) results2=model1.fit() print(results2.summary())
#http://barnesanalytics.com/basics-of-arima-models-with-statsmodels-in-python